# Original problem
- You had a base Logger class that would log messages to a file natively.
- If you wanted to log messages to other destinations such as a socket, syslog etc, you had to create separate subclasses for each destination.
- This would lead to a proliferation of subclasses and code duplication.
- Making it harder to maintain the classes. And if you wanted to add new features, you would have to create even more subclasses.

In [1]:
# original Logger class
class Logger:
    def __init__(self, file):
        self.file = file
    def log(self, message):
        self.file.write(message + '\n')
        self.file.flush()


# adding a new output destination (e.g., socket)
class SocketLogger(Logger):
    def __init__(self, socket):
        self.socket = socket
    def log(self, message):
        self.socket.sendall((message + '\n').encode('utf-8'))


# adding another output destination (e.g., database)
class DatabaseLogger(Logger):
    def __init__(self, db_connection):
        self.db_connection = db_connection
    def log(self, message):
        cursor = self.db_connection.cursor()
        cursor.execute("INSERT INTO logs (message) VALUES (?)", (message,))
        self.db_connection.commit()

# Solution 1 ✨ - Adapter Pattern
- Instead of creating many different sub-classes for each logging destination, you can create an Adapter which will make the different logging destinations look like the same file interface the original Logger class expects.
- So, now, you can have the single Logger class and create multiple adapters for different logging destinations.

## Duck typing
Adapter pattern relies on duck typing which means that if an object behaves like a certain type (by having the same methods and attributes), it can be treated as that type, regardless of its actual class (and type and class hierarchy are not checked).

In [4]:
# creating adapter for socket class
class SocketAdapter:
    def __init__(self, socket):
        self.socket = socket
    def write(self, message):
        self.socket.sendall((message + '\n').encode('utf-8'))
    def flush(self):
        pass


In [ ]:
import sys

logger = Logger(sys.stdout)
logger.log("This is a log message to standard output.")

# creating a socket adapter
socket_logger = Logger(SocketAdapter(socket))
socket_logger.log("This is a log message to socket.")